This notebook contains Python equivalent code to the example in: https://gitlab.stimulate.ovgu.de/christoph.reichert/visual-spatial-attention-bci

The notebook should produce close enough data that should be inter-changeable with the resulting data from the original
Matlab script.

How to use this:

1. Create a folder which will contain only dataset files (default name is "dataset" in the same folder as this notebook)

2. Run all code blocks in this notebook

3. The notebook will generate output files in output folder (default name is "dataset_result")

In [ ]:
import scipy.signal
import numpy as np
from pymatreader import read_mat
import os
from sklearn.cross_decomposition import CCA

In [ ]:
# Create a list with all the dataset files
dataset_files = []
dataset_folder = 'dataset' # note that this folder

for file in os.listdir(dataset_folder):
    dataset_files.append(os.path.join(dataset_folder, file)) # append relative path

In [ ]:
def preprocess_data(mat):
    bci_exp = mat['bciexp'] # reference to the bci exp data which are the only relevant data
    labels = bci_exp['label'] # all channels

    sampling_rate = bci_exp['srate']
    downsampling_fact = 5 # downsampling factor, 250 / 5 = 50 Hz
    bandpass = np.array([1, 12.5]) # cutoff frequencies for the bandpass filter
    interval_len = .75 # seconds

    # calculate bandpass filter coefficients
    butter_b, butter_a = scipy.signal.butter(N=2, Wn=bandpass / (sampling_rate / 2), btype='bandpass')

    channels_of_interest = ['O9', 'CP1', 'CP2', 'O10', 'P7', 'P3', 'Pz',
            'P4', 'P8', 'PO7', 'PO3', 'Oz', 'PO4', 'PO8']

    # get the index of each channel in labels array
    channel_indexes = np.array([labels.index(ch) for ch in channels_of_interest])
    lmast_channel_idx = np.char.equal('LMAST', labels)

    # number of samples per analysis window
    num_samples_per_window = int(interval_len * sampling_rate / downsampling_fact) - 1

    stimuli = np.array(bci_exp['stim'], dtype=np.double)
    num_stimuli = np.sum(np.diff( np.sum(stimuli[:, :, 0], axis=0)) > 0)

    eeg_data = bci_exp['data']
    num_trials = eeg_data.shape[2]
    num_channels = len(channel_indexes)
    data = np.zeros(shape=(num_channels, num_samples_per_window, num_stimuli, num_trials))
    model = np.zeros(shape=(num_samples_per_window, num_samples_per_window, num_stimuli, num_trials))

    # For each trial
    for tr in range(num_trials):
        rdat = eeg_data[channel_indexes, :, tr] - (eeg_data[lmast_channel_idx, :, tr] / 2)
        filtfilt_signal = scipy.signal.filtfilt(butter_b, butter_a, rdat, padtype='odd',
                                                padlen=3 * (max(len(butter_b),len(butter_a)) - 1))
        rdat = scipy.signal.resample_poly(filtfilt_signal, 1, downsampling_fact, axis=1)
        rdat = rdat.T
        x1 = np.insert(stimuli[0, :, tr], 0, 0, axis=0)
        x2 = np.insert(stimuli[1, :, tr], 0, 0, axis=0)
        stim = np.array((np.diff(x1) > 0) + (np.diff(x2) > 0), dtype=np.double)

        stim_onsets = np.array(np.where(stim != 0))[0]

        for st in range(num_stimuli):
            start = int(np.ceil((stim_onsets[st] + 1) / downsampling_fact)) - 1
            idx = np.arange(start, start + num_samples_per_window)

            data[:, :, st, tr] = rdat[idx, :].T
            model[:, :, st, tr] = stim[stim_onsets[st]] * np.eye(num_samples_per_window)

    return data


In [ ]:
def apply_spatial_filter(data, model, labels):
    num_channels = data.shape[0]
    num_trial_samples = data.shape[1] * data.shape[2]

    X = np.zeros(shape=(num_trial_samples * data.shape[3], data.shape[0]))
    Y = np.zeros(shape=(num_trial_samples * model.shape[3], model.shape[0]))

    idx = np.arange(0, 1)
    # Concatenate trials
    for trial in range(data.shape[3]):
        idx = np.arange(idx[-1], num_trial_samples)
        X[idx, :] = np.reshape(np.transpose(data[:, :, :, trial], axes=(2, 3, 1)),
                               newshape=(num_trial_samples, num_channels))

        y_sign = 1 if labels[trial] == 'yes' else -1
        Y[idx, :] = y_sign * np.reshape(np.transpose(model[:, :, :, trial], axes=(2, 3, 1)),
                                        newshape=(num_trial_samples, model.shape[0]))

    T = {}

[ 0  1  2  3  4  5  6  7  8  9 10 11]
